In [ ]:
import glob
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import os

import scanpy as sc
import seaborn as sns

from scroutines import basicu

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import ValueWarning
from tqdm import tqdm

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/annot/gencode.vM25.genebody.bed'
df_annot = pd.read_csv(f, sep='\t', header=None)
df_annot

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/DEG_l23abc_gene_list_250409.csv'
df_annot_abc = pd.read_csv(f)
df_annot_abc['cond'] = df_annot_abc['cond'].apply(lambda x: x.replace('NR', ''))
df_annot_abc = df_annot_abc.groupby('gene').agg({
    'cond': lambda x: ' '.join(x), 
    'archetype': lambda x: ''.join(np.unique(x)), 
    })
df_annot_abc

In [ ]:
ddir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_nrdr_lmm/'
!ls $ddir

In [ ]:
df_sig_all = [] 
for tag in ['A', 'B', 'C', 'ALL']:
    filename = os.path.join(ddir, f'NRDR_DEGs_LMM_P21_L23_{tag}.csv')
    df = pd.read_csv(filename, index_col=0)
    df['max_cp10k'] = np.max(df.iloc[:,5:], axis=1)
    x = df['log2fc']
    y = df['qval']
    z = df['max_cp10k']
    # z2 = df['converged']

    x_th = np.log2(1.5)
    y_th = 0.05
    z_th = 0.1

    cond_sig = np.all([np.abs(x) > x_th, y < y_th, z > z_th,], axis=0)
    
    print(np.sum(cond_sig))
    df_sig = df[cond_sig].copy().sort_values('qval')
    df_sig = df_sig.set_index('gene').join(df_annot_abc)
    df_sig['test_sig'] = tag
    df_sig = df_sig[['cond', 'archetype', 'test_sig']]
    df_sig_all.append(df_sig)
    
df_sig_all = pd.concat(df_sig_all)
# df_sig_all = df_sig_all.reset_index().groupby('gene').agg({'test_sig': lambda x: ' '.join(x)})
df_sig_all

In [ ]:
for tag in ['A', 'B', 'C', 'ALL']:
    filename = os.path.join(ddir, f'NRDR_DEGs_LMM_P21_L23_{tag}.csv')
    df = pd.read_csv(filename, index_col=0)
    print(tag)
    print(df.set_index('gene').loc[
        # ['Syt7', 'Amph', 'Rimbp2', 'Rapgef4', 'Dtna']
        # ['Sema6a', 'Pcdh15']
        ['Slit3']
    ])

In [ ]:
filename = os.path.join(ddir, f'NRDR_DEGs_LMM_P21_L23_C.csv')
df = pd.read_csv(filename, index_col=0)
df.set_index('gene').loc['Scg2']

In [ ]:
df_sig_all_summary = df_sig_all.reset_index().groupby(['gene', 'test_sig']).size().unstack()
df_sig_all_summary

In [ ]:
# df_sig_all_summary.loc['Fosl2']

In [ ]:
# df_sig_abc = df_sig.set_index('gene').join(df_annot_abc).dropna().sort_values('archetype')
df_sig_all_abc = df_sig_all.dropna() #.sort_values('archetype')# .dropna()
df_sig_all_abc = df_sig_all_abc.reset_index().groupby('gene').agg({'cond': 'first', 'archetype': 'first', 'test_sig': lambda x: " ".join(x)})  #.sort_values('archetype')
print(df_sig_all_abc.shape, len(df_sig_all_abc)/len(df_sig_all))
df_sig_all_abc

In [ ]:
df_sig_all_abc.sort_values('archetype').to_csv('check_l23nrdr.csv')

output = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_sexual_dimorphism/summary_l23_p21nrdr.csv'
df_sig_all_abc.sort_values('archetype').to_csv(output)

df_sig_all_abc.sort_values('archetype') # shape

In [ ]:
df_sig_all_abc.value_counts('archetype')

In [ ]:
df_sig_all_abc.value_counts('archetype').plot.bar()

In [ ]:
df_sig_all_abc.value_counts('test_sig')

In [ ]:
df_sig_all_abc.sort_values('test_sig')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, -np.log10(y), s=5)
# for idx, row in df[cond_sig].iterrows():
#     ax.text(row['log2fc'], -np.log10(row['qval']), row['gene'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x, -np.log10(y), s=5)
ax.set_ylim([-1,10])
# for idx, row in df[cond_sig].iterrows():
#     ax.text(row['log2fc'], -np.log10(row['qval']), row['gene'])
plt.show()

In [ ]:
2**(-0.59)

In [ ]:
np.log2(1.5)